In [27]:
import requests
import os
from xml.etree import ElementTree 
import pandas

from typing import Dict
from typing import Tuple
from typing import Optional
from typing import List


host_name = "www.deribit.com"
deribit_endpoint_v2 = f"https://{host_name}/api/v2"
public_url = f"{deribit_endpoint_v2}/public"
private_url = f"{deribit_endpoint_v2}/private"



def authorize_with_credentials(client_id: str, client_secret: str) -> Tuple[Dict[str, str], str]:
    print(f"connecting with {client_id}")
    headers = {"Content-Type": "application/json"}
    auth_params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials"
    }
    authentication = requests.get(f"{public_url}/auth", params=auth_params, headers=headers).json()
    headers["Authorization"] = f"Bearer {authentication['result']['access_token']}"
    refresh_token = authentication['result']['refresh_token']
    return headers, refresh_token

def authorize_refresh(refresh_token: str) -> Tuple[Dict[str, str], str]:
    print(f"refreshing authentication {refresh_token}")
    headers = {"Content-Type": "application/json"}
    auth_params = {
        "refresh_token": refresh_token,
        "grant_type": "refresh_token"
    }
    authentication = requests.get(f"{public_url}/auth", params=auth_params, headers=headers).json()
    headers["Authorization"] = f"Bearer {authentication['result']['access_token']}"
    new_refresh_token = authentication['result']['refresh_token']
    return headers, new_refresh_token

def run_get(target: str, params: Dict[str, str], headers: Dict[str, str], refresh_token: str=None) -> Optional[Tuple[Dict[str, str], str]]:
    response = requests.get(target, params=params, headers=headers)
    if response.status_code >= 399:
        error = response.json()
        if error['error']['code'] == 13009 and refresh_token:
            # re-run authorization
            new_headers, refresh_token = authorize_refresh(refresh_token)
            response = requests.get(target, params, new_headers)
        else:
            return None

    return response.json()['result'], headers, refresh_token


def load_ib_positions(ib_token: str, ib_query_id: str) -> List[Dict[str, str]]:
    ib_flex_url = f"https://www.interactivebrokers.com/Universal/servlet/FlexStatementService.SendRequest?t={ib_token}&q={ib_query_id}&v=3"
    ib_response = requests.get(ib_flex_url)
    ib_report_location = ElementTree.fromstring(ib_response.content)
    ib_report_reference_code = ib_report_location.find('ReferenceCode').text
    ib_report_base_url = ib_report_location.find('Url').text
    ib_report_url = f"{ib_report_base_url}?t={ib_token}&q={ib_report_reference_code}&v=3"
    ib_report = requests.get(ib_report_url)
    ib_report_content = ElementTree.fromstring(ib_report.content)
    ib_positions = []
    for position in ib_report_content.findall('.//OpenPosition'):
        ib_positions.append(position.attrib)


connecting with 7XMV2uf-


In [28]:

headers, refresh_token = authorize_with_credentials(client_id=os.getenv("DERIBIT_CLIENT_ID"), client_secret=os.getenv("DERIBIT_CLIENT_SECRET"))

account_summary_deribit = {}
for currency in ("BTC", "ETH", "USDT", "USDC"):
    account_summary_params = {"currency": currency}
    result, headers, refresh_token = run_get(f"{private_url}/get_account_summary", params=account_summary_params, headers=headers, refresh_token=refresh_token)
    account_summary_deribit[currency] = result

account_summary_deribit


{'BTC': {'margin_model': 'legacy_pm',
  'delta_total_map': {'btc_usd': -110.57877343},
  'options_vega': -14543.25087,
  'session_upl': 0.44350842,
  'projected_delta_total': -102.186295,
  'options_gamma': -0.00229,
  'options_theta_map': {'btc_usd': 3684.37806},
  'total_pl': -73.34019261,
  'projected_initial_margin': 44.71861443,
  'available_withdrawal_funds': 72.04526385,
  'deposit_address': 'bc1qw8rj57p23gwq70s87h49p8j8hs0gzy7xla4yxl',
  'limits': {'non_matching_engine': {'burst': 100, 'rate': 20},
   'matching_engine': {'burst': 30, 'rate': 10}},
  'options_delta': -110.51411,
  'options_pl': -73.29545676,
  'futures_session_upl': 0.00029502,
  'portfolio_margining_enabled': True,
  'delta_total': -102.186295,
  'maintenance_margin': 36.46824671,
  'cross_collateral_enabled': False,
  'futures_pl': -0.04473585,
  'session_rpl': 0.0,
  'options_value': -8.39247874,
  'projected_maintenance_margin': 36.46824671,
  'options_gamma_map': {'btc_usd': -0.00229},
  'options_session_rp

In [63]:
ib_token = os.getenv("IB_FLEX_REPORT_TOKEN")
ib_query_id = os.getenv("IB_FLEX_QUERY_ID")
ib_positions = load_ib_positions(ib_token, ib_query_id)
pandas.DataFrame(ib_positions)


,accountId,acctAlias,model,currency,fxRateToBase,assetCategory,subCategory,symbol,description,conid,...,vestingDate,code,originatingOrderID,originatingTransactionID,accruedInt,serialNumber,deliveryType,commodityType,fineness,weight
0,U11301093,Investo Multi CH1234846454 U11301093,,EUR,0.93058,STK,COMMON,STLA,STELLANTIS NV,463647471,...,,,,,,,,,0.0,0.0
1,U11301093,Investo Multi CH1234846454 U11301093,,HKD,0.10888,STK,COMMON,1263,PC PARTNER GROUP LTD,100197655,...,,,,,,,,,0.0,0.0
2,U11301093,Investo Multi CH1234846454 U11301093,,HKD,0.10888,STK,COMMON,1419,HUMAN HEALTH HOLDINGS LTD,227795491,...,,,,,,,,,0.0,0.0
3,U11301093,Investo Multi CH1234846454 U11301093,,HKD,0.10888,STK,COMMON,746,LEE & MAN CHEMICAL COMPANY L,42451353,...,,,,,,,,,0.0,0.0
4,U11301093,Investo Multi CH1234846454 U11301093,,HKD,0.10888,STK,COMMON,867,CHINA MEDICAL SYSTEM HOLDING,95547522,...,,,,,,,,,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,U8361124,Bonds CH168858350 U8361124,,USD,0.85022,BOND,,RCL 9 1/4 01/15/29 TAK9,RCL 9 1/4 01/15/29,588071754,...,,,,,4445.14,,,,0.0,0.0
228,U8361124,Bonds CH168858350 U8361124,,USD,0.85022,BOND,,TDG 4 5/8 01/15/29 7BP1,TDG 4 5/8 01/15/29,514232238,...,,,,,2222.57,,,,0.0,0.0
229,U8361124,Bonds CH168858350 U8361124,,USD,0.85022,BOND,,TEVA 4.1 10/01/46,TEVA 4.1 10/01/46,241553875,...,,,,,1104.72,,,,0.0,0.0
230,U8361124,Bonds CH168858350 U8361124,,USD,0.85022,BOND,,THC 4 7/8 01/01/26 GCZ1,THC 4 7/8 01/01/26,609827371,...,,,,,94.79,,,,0.0,0.0
